<h1 style="color:black" align="center">Интерпретация моделей</h1>

<h1 style="color:#008B8B">1. Что находят свёрточные сети?</h1>

https://cs.nyu.edu/~fergus/papers/zeilerECCV2014.pdf

* Можно для каждого фильтра найти кусочки картинок, дающие самый сильный отклик
* Сделаем это для AlexNet

Можно среди всей обучающей выборки найти картинку и кусочек на ней, где фильтр дает наибольший отклик. Посомтрим на пример 3-го сверточного слоя:

<img src='img/lecture07/1.png'>

## Максимизация вероятности класса

### Вариант 1

https://arxiv.org/pdf/1312.6034.pdf

Возьмем обученную нейронную сеть $a_y(x)$, где $y$ - это выход конкретного класса, например, вероятность того, что это кошка. Дальше подаем на вход случайную картинку $x$, после чего начинаем изменять данную картинку градиентным спуском, чтобы максимизировать вероятность того, что на картинке кошка. Мы же можем градиентным спуском обучать не параметры, а саму картинку. Тогда посчитам backprop выходы сетки по пикселям входной картинки, после чего сдвигать пиксели, чтобы вероятность максимизировалась.

Так же, имеется регуляризатор, который ограничивает величины интенсивности цветов, которые записаны в пикселях.

$\large a_y(x) - \lambda \|x\|_2^2 \to \underset{x}{\text{min}}$

Посмотрим, какие картинки с точки зрения нейронной сети с большей вероятностью содержат тот или иной класс:

<img src='img/lecture07/2.png'>

### Вариант 2

Здесь улучшенная версия

https://ai.googleblog.com/2015/06/inceptionism-going-deeper-into-neural.html

<img src='img/lecture07/3.png'>

Давайте теперь инициализировть вход не случайным шумом, как это было на прошлых картинках, а возьмем за основу некоторую картинку и будем максимизировать вероятность некоторого класса.

<img src='img/lecture07/4.png'>

Дальше можно взять большие картинки и максимизировать вероятность некоторого класса.

<img src='img/lecture07/5.png'>

<h1 style="color:black" align="center">Компьютерное зрение - Сегментация</h1>

Мы научились выполнять классификацию изображений с использованием нейронных сетей.

<img src='img/lecture07/6.png'>

**Зачастую, хочется решать другие задачи:**

* Semantic Segmentation - какой пиксель картинки относится к какому классу. Желтый пиксел - это класс кот, фиоллетовый пиксель - это лес...

* Object detection - не просто говорим что собака находится на изображении, а говорим где она находится, выделив их прямоугольниками.

* Instanct Segmentation - в этом примере мы выделяем область на картинге, где имеется собака.

<h1 style="color:#008B8B">1. Семантическая сегментация</h1>

Все задачи выше мы будем сводить к классификации, так как мы уже умеем решать данную задачу.

<img src='img/lecture07/7.png'>

В этой задаче необходимо для каждого пикселя картинки сказать, к какому классу этот пиксель относится.

<h2 style="color:#008B8B">1.1 Постановка задачи</h2>

* Данные: изображения и их корректные сегментации
* Пример: PASCAL VOC 2012

Изначально, необходимо разобраться как устроены данные.

<img src='img/lecture07/11.png'>

Как видим, у нас имеется весьма мало объектов внутри классов, как мы будем это обучать на таком маленьком наборе данных? Поскольку каждая картинка несет в себе дополнительную информацию, так как класс известен для каждого пикслея. В случае, если у нас $1500$ картинок размером $100 \times 100$, тогда всего пикселей, для которых известны классы $15,000, 000$. 

Следовательно, в случае сегментации мы можем работать с очень маленькими выбороками, так как мы решаем задачу классификации не картинок, а классификации каждого пикселя.

* Каждый объект содержит сильно больше информации, чем при
классификации!
* Можно хорошо обучаться на небольших выборках

<h2 style="color:#008B8B">1.2 Метрики качества</h2>

### Попиксельная доля верных ответов:

$$\large L(y, a) = \frac{1}{n} \sum\limits_{i=1}^n [y_i = a_i]$$

Где $n$ - число пикселей в картинке. $y_i$ правильный класс для $i$-го пикслея, $a_i$ класс, который выдала модель для $i$-го пикселя.

Суммируем по всем пикселям и смотрим долю пикселей где модель угадала правильный класс. Обучаться модель будет на другой функционал, так как данный функционал не дифференцируемый.

### Мера Жаккара (считается отдельно для каждого класса):

Если почти всю картинку занимает автобус, но модель захватила весь автобус и несколько областей рядом. Скорее всего в этом нет ничего страшного, так как автобус был найден и мы не хотим штрафовать модель за небольшие отклонения от правильного сегмента. Для этого используется Мера Жаккара (IoU): 

$$\large J_k (y, a) = \frac{\sum_{i=1}^n [y_i = k] [a_i = k]}{\sum_{i=1}^n \text{max}([y_i = k], [a_i = k])}$$


Мера Жаккара считается для каждого класса в отдельности. Например, мы взяли калсс автобус и проходимся по всем пикселям. В числителе считаем число пикселей, которые должны относиться к автобусу и модель отнесла их к автобусу. В знаменателе считаем число пикселей, которые на самом деле относятся к автобусу или модель отнесла к автобусу.

Другими словами, мы берем две области. Первая область, где модель отнесла к автобусу и где нас самом деле находится автобус. В числителе мы считаем площадь пересечения двух областей и делим на площадь объеденения двух областей. Можно посчитать меру Жаккара для каждого класса и усреднить по всем классам.

<h2 style="color:#008B8B">1.3 Функция потерь</h2>

### Для одного изображения (categorical cross-entropy, CCE):

<img src='img/lecture07/12.png'>

Просуммируем по всем пикселям картинки, в каждом пикселе просуммируем по всем классам (например, 20 различных классов, автобус, машина...). Берем идикатор того, что $y_i$ пиксель относится к $k$ классу (так как пиксель может относится только к одному классу) и умножаем на логарифм $a_{ik}$ вероятности $k$-го класса на $i$ пикселе. 

И все это мы будем максимизировать, тем самым требуя, чтобы вероятность правильного класса в каждом пикселе была как можно больше, значит и хотим, чтобы распределение вероятностей было вырожденным. Можно поставить минус перед функцией потерь и минимизировать функцию потерь. В логистической регрессии была подобная запись, только здесь записано для случия многих классов. 

$a_{ik}$ - Это выход модели в $i$-ом пикселе для $k$-го класса. Мы считаем, что модель $a(x)$ принимает на вход картинку и если исходная картинка $n \times m \times c$, тогда модель вернет картинку $n \times m \times K$. Сдовательно, на выходе модели записана вероятность каждого из классов. Мы считаем, что модель должна для каждого пикселя выдать $K$ вероятностей (распределение классов в этих точках).

**Почему берем логарифм?** Вспомним проблему сигмойды, что происходит затухание градиента. Засчет того, что мы используем логарифм, решается проблема насыщения. Поскольку $a_{ik}$ - это softmax (может происходить насщение), логарифмом мы получаем функцию, которая решает эту пробему. Ну и при этом, можно интерпретировать данную запись как правдоподобие... 

### SoftMax

Возникает вопрос, как получить вероятности? Имеется некоторая архитектура нейронной сети, которая, например, после полносвязного слоя для каждого пикселя выдает вектор из $K$ чисел. А нам хочется получать вероятности, чтобы они суммировались в еденицу и чтобы они все были неотрицательны. И набор из $K$ числе превратить в набор вероятностпй можно использованием softmax.

Если модель в $i$-м пикселе выдаёт какие-то числа $b_{i1}, \ldots, b_{iK}$, то их можно превратить в вероятности через softmax:

$$\large a_{ik} = \frac{\exp(b_{ik})}{\sum_{m=1}^k \exp(b_{im})}$$

Например, для какого пикслея на выходе получилось $K$ числе на выходе: $b_{i1}, \ldots, b_{iK}$, тогда возьмем от каждого из чисел экспоненту и разделим на сумму экспонент.

Важно, что softmax просто преобразует выходы. А вероятности получаются засчет того, что мы обучаем на подобранный функционал.

<h1 style="color:#008B8B">2. Fully Convolutional Network</h1>

Fully Convolutional Network - означает, что в нейронной сети нет полносвязных слоев, в ней только сверточные слои, max pooling.

https://arxiv.org/abs/1411.4038

Начнем с первого подхода к сегментации изображений. Имеется картинка слева и мы хотим получить карту сегментации справа, где для каждого пикселя сказано к какому классу он относится.

Возьмем сверточную нейронную сеть VGG/AlexNet... мы помним, что данные архитектуры представляют собой последовательность светочных слоев с последующими полносвязными слоями на концах. В архитектуре оставим только сверточную часть нейронной сети убрав полносвязные слои. В конце такой архитектуры после последнего сверточного слоя на выходе получается некоторый тензор небольшого размера (определнной ширины и высоты) с 21 каналом. Если изначально мы начинали с картинки размером $200 \times 200 \times 3$, то в конце получается картинка, например, размера $20 \times 20 \times 21$ и эту картинку необходимо расширить в картинку исходного размера с 21 каналом ($200 \times 200 \times 21$). И к этому применяем полносвязный слой, чтобы превратить полученные каналы в вероятности классов для каждого пикселя.

Замечание, 21 канал получился не просто так. Так как мы делаем количество каналов в последенм сверточном слое столько, сколько классов. 

<img src='img/lecture07/8.png'>

**Итого:**

* Убираем полносвязные слои
* Остаются только свёртки
* Тензор с последнего слоя преобразуем свёртками 1x1 в тензор такого же размера, но с $K$ каналами (по числу классов)
* Повышаем разрешение
    * Можно простым размножением пикселей
    * Можно хитрее
    
### Пример работы

FCN-xxs - число характеризует архитектуру, чем меньше число, тем больше архитектура. Как видим даже самая большая архитектура плохо работает. 

<img src='img/lecture07/9.png'>

**Чем это плохо?**

* Тензор на последнем слое довольно маленький
* Классы предсказываются грубо, у объектов нечёткие границы

Почему это работает плохо? Так как мы сжали огромную картинку путем сверток и max pooling до небольшого размера, потеряв много информации. У нас информации мало, она сжатая и обобщенная, так как мы потеряли знание об отдельных пикселях. А после мы все это разворачиваем в большую картинку и пытаемся классифицировать пиксели.

**Замечание:** В нашей архитектуре у выхода последнего сверточного слоя с 21 каналом большое поле восприятие, следовательно каждый фильтр видит всю картинку и мы выполняем сегментацию опираясь на полную информацию о картинке. Если взять только первый сверточный слой, который имеет большой размер и по нему построим сегментацию, тогда это будет полохо, так как у каждого фильтра маленькое поле восприятия. Следовательно, мы принимаем решение о том, к какому классу относится пикслеь только на основе небольшой области, что плохо.

Делаем вывод, что нам необходимо балансироваться между полем восприятия и размером картинки, что весьма сложно. Поэтому придумали другие архитектуры, которые мы рассмотрим ниже.

**Решение:**

* Можно делать меньше пулингов
* Но тогда будут проблемы с размером поля восприятия

<h1 style="color:#008B8B">3. U-Net</h1>

https://arxiv.org/abs/1505.04597

**Шаг 1.** Имеется входная картинка размером $572 \times 572 \times 1$ и выполняем две операции свертки с фильтром размера $3 \times 3$ с последующим применением ReLU после каждой свертки.  Заметим, что после сверток изображение уменьшается в размере, так как не используется Padding.

**Шаг 2.** Используем max pool $2 \times 2$ и повторяем операцию свертки. И так несколько раз...

**Шаг 3.** Получаем после очередного max pooling картинку размером $32 \times 32 \times 512$. После выполняем две операции всертки подряд и на выходе получаем изображение небольшого размера $28 \times 28 \times 1024$

<img src='img/lecture07/13.png'>

**Шаг 4.** Применяем операцию up-conv $2 \times 2$ которая увеличивает входное изображение $28 \times 28 \times 1024$ (Тензор 1) в два раза некоторым образом и подгоняем количество каналов как в левой ветке (Зеленая стрелочка). После преобразований получаем картинку размером $56 \times 56 \times 512$ (Тензор 2). Тензор изображений из левой ветки размером $64 \times 64 \times 512$ обрезаем до $56 \times 56 \times 512$, после эту преобразованную картинку из левой ветки добавяем в качестве каналов к основной картинке (3 тензор).

**Шаг 5.** Выполняем такую опреацию несколько раз.

**Шаг 6.** На последней серой стрелочке мы берем исходную картинку без использования пулинга, соответственно мы имеем исходную информацю о каждом пикселе. Применяем два раза свертки и пследнюю свертку с фильтром $1 \times 1$, которая сжимает все каналы в такокое количество - сколько у нас классов. 

После получаем карту сегментации, которая представляет собой картинку исходного размера, где для каждого пикселя сказано, к какому классу он относится.